### Importing Required Libraries

In [1]:
# importing required libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error as msle

### Reading Data

In [2]:
# reading the dataset
train_data = pd.read_csv('Data/updated_train_data.csv')
product_data = pd.read_csv('Data/updated_product_data.csv')
store_data = pd.read_csv('Data/updated_store_data.csv')

In [3]:
train_data.head()

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS
0,14-Jan-09,367,1111009477,1.57,0,0,13
1,14-Jan-09,367,1111009497,1.39,0,0,20
2,14-Jan-09,367,1111085319,1.88,0,0,14
3,14-Jan-09,367,1111085345,1.88,0,0,29
4,14-Jan-09,367,1111085350,1.98,0,0,35


In [4]:
product_data.head()

,UPC,MANUFACTURER_1,MANUFACTURER_2,MANUFACTURER_3,MANUFACTURER_4,MANUFACTURER_5,MANUFACTURER_6,MANUFACTURER_7,MANUFACTURER_8,MANUFACTURER_9,...,CATEGORY_3,CATEGORY_4,SUB_CATEGORY_1,SUB_CATEGORY_2,SUB_CATEGORY_3,SUB_CATEGORY_4,SUB_CATEGORY_5,SUB_CATEGORY_6,SUB_CATEGORY_7,PRODUCT_SIZE
0,1111009477,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,1111009497,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
2,1111009507,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
3,1111038078,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1111038080,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [5]:
store_data.head()

,STORE_ID,ADDRESS_STATE_PROV_CODE_1,ADDRESS_STATE_PROV_CODE_2,ADDRESS_STATE_PROV_CODE_3,ADDRESS_STATE_PROV_CODE_4,MSA_CODE_1,MSA_CODE_2,MSA_CODE_3,MSA_CODE_4,MSA_CODE_5,MSA_CODE_6,MSA_CODE_7,MSA_CODE_8,MSA_CODE_9,SEG_VALUE_NAME,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,367,1,0,0,0,1,0,0,0,0,0,0,0,0,1,24721,12707
1,389,1,0,0,0,1,0,0,0,0,0,0,0,0,2,46073,24767
2,613,0,1,0,0,0,1,0,0,0,0,0,0,0,2,64926,29386
3,623,0,1,0,0,0,0,1,0,0,0,0,0,0,2,46930,36741
4,2277,0,0,1,0,1,0,0,0,0,0,0,0,0,3,81958,54053


#### Merging 3 datasets into 1 dataset

In [6]:
merged_data = train_data.merge(product_data, how= 'left', on= 'UPC')
merged_data = merged_data.merge(store_data,how='left', left_on = 'STORE_NUM', right_on='STORE_ID')

In [7]:
# convert to datetime
merged_data.WEEK_END_DATE = pd.to_datetime(merged_data.WEEK_END_DATE)

# create an array of unique week dates
week = merged_data.WEEK_END_DATE.unique()

In [8]:
original_train_data = pd.read_csv('Data/train.csv', usecols= ['STORE_NUM','UPC'])

In [9]:
original_product_data = pd.read_csv('Data/product_data.csv')

In [10]:
original_train_data = original_train_data.merge(original_product_data, how='left',on = 'UPC')

In [11]:
# Now, we will create another dataframe unique_store_data
# We will group the data by STORE_NUM and find unique values of the features 'MANUFACTURER', 'CATEGORY', 'SUB_CATEGORY'

unique_store_data = original_train_data.groupby(['STORE_NUM'])['MANUFACTURER', 'CATEGORY', 'SUB_CATEGORY'].nunique().reset_index()

In [12]:
# rename the columns of the dataframe 'unique_store_data'
unique_store_data.columns = ['STORE_NUM', 'U_MANUFACTURER', 'U_CATEGORY', 'U_SUB_CATEGORY']

In [13]:
# now , merge this new dataframe with the dataframe that has all the features.
data_with_unique_store = merged_data.merge(unique_store_data, how= 'left', on= 'STORE_NUM')

In [14]:
data_with_unique_store.head()

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS,MANUFACTURER_1,MANUFACTURER_2,MANUFACTURER_3,...,MSA_CODE_6,MSA_CODE_7,MSA_CODE_8,MSA_CODE_9,SEG_VALUE_NAME,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS,U_MANUFACTURER,U_CATEGORY,U_SUB_CATEGORY
0,2009-01-14,367,1111009477,1.57,0,0,13,1,0,0,...,0,0,0,0,1,24721,12707,4,3,5
1,2009-01-14,367,1111009497,1.39,0,0,20,1,0,0,...,0,0,0,0,1,24721,12707,4,3,5
2,2009-01-14,367,1111085319,1.88,0,0,14,1,0,0,...,0,0,0,0,1,24721,12707,4,3,5
3,2009-01-14,367,1111085345,1.88,0,0,29,1,0,0,...,0,0,0,0,1,24721,12707,4,3,5
4,2009-01-14,367,1111085350,1.98,0,0,35,1,0,0,...,0,0,0,0,1,24721,12707,4,3,5


In [15]:
data_with_unique_store.columns

Index(['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'BASE_PRICE', 'FEATURE', 'DISPLAY',
       'UNITS', 'MANUFACTURER_1', 'MANUFACTURER_2', 'MANUFACTURER_3',
       'MANUFACTURER_4', 'MANUFACTURER_5', 'MANUFACTURER_6', 'MANUFACTURER_7',
       'MANUFACTURER_8', 'MANUFACTURER_9', 'CATEGORY_1', 'CATEGORY_2',
       'CATEGORY_3', 'CATEGORY_4', 'SUB_CATEGORY_1', 'SUB_CATEGORY_2',
       'SUB_CATEGORY_3', 'SUB_CATEGORY_4', 'SUB_CATEGORY_5', 'SUB_CATEGORY_6',
       'SUB_CATEGORY_7', 'PRODUCT_SIZE', 'STORE_ID',
       'ADDRESS_STATE_PROV_CODE_1', 'ADDRESS_STATE_PROV_CODE_2',
       'ADDRESS_STATE_PROV_CODE_3', 'ADDRESS_STATE_PROV_CODE_4', 'MSA_CODE_1',
       'MSA_CODE_2', 'MSA_CODE_3', 'MSA_CODE_4', 'MSA_CODE_5', 'MSA_CODE_6',
       'MSA_CODE_7', 'MSA_CODE_8', 'MSA_CODE_9', 'SEG_VALUE_NAME',
       'SALES_AREA_SIZE_NUM', 'AVG_WEEKLY_BASKETS', 'U_MANUFACTURER',
       'U_CATEGORY', 'U_SUB_CATEGORY'],
      dtype='object')

### Function to create Validation Sets

In [16]:
from datetime import timedelta
def validation_df(data, week, no_of_months, no_of_validation):
    
    model_set = []
    set_n = 1
    for w in range(len(week)-1,0,-1):
        x_data = {}

        x_data['train_start'] = week[w-3-4*no_of_months]
        x_data['train_end'] = week[w-4]
        x_data['validate_week'] = week[w-2]
        x_data['test_week'] = week[w]
        x_data['no_days_train'] = x_data['train_end'] - x_data['train_start']
        x_data['set_no'] = 'set'+str(set_n)
        set_n +=1
        model_set.append(x_data)
        if(len(model_set) == no_of_validation):
            break
        
        
    datapoints = []

    for s in model_set :
        x = {}
        train_set = data[(data.WEEK_END_DATE >= s['train_start']) & (data.WEEK_END_DATE <= s['train_end'])]
        x['train_shape'] = train_set.shape[0]
        x['validation_shape']  = data[data.WEEK_END_DATE == s['validate_week']].shape[0]
        x['test_shape'] = data[data.WEEK_END_DATE == s['test_week']].shape[0]
        x.update(s)
        datapoints.append(x)

    df = pd.DataFrame.from_dict(datapoints)
    df['no_days_train'] = df['no_days_train'] + timedelta(days=7)
    return df    

In [17]:
validation_df(merged_data, week, no_of_months=2, no_of_validation=14)

,train_shape,validation_shape,test_shape,train_start,train_end,validate_week,test_week,no_days_train,set_no
0,13089,1640,1642,2011-07-13,2011-08-31,2011-09-14,2011-09-28,56 days,set1
1,13102,1632,1638,2011-07-06,2011-08-24,2011-09-07,2011-09-21,56 days,set2
2,13101,1629,1640,2011-06-29,2011-08-17,2011-08-31,2011-09-14,56 days,set3
3,13101,1640,1632,2011-06-22,2011-08-10,2011-08-24,2011-09-07,56 days,set4
4,13109,1638,1629,2011-06-15,2011-08-03,2011-08-17,2011-08-31,56 days,set5
5,13108,1631,1640,2011-06-08,2011-07-27,2011-08-10,2011-08-24,56 days,set6
6,13108,1640,1638,2011-06-01,2011-07-20,2011-08-03,2011-08-17,56 days,set7
7,13112,1637,1631,2011-05-25,2011-07-13,2011-07-27,2011-08-10,56 days,set8
8,13104,1635,1640,2011-05-18,2011-07-06,2011-07-20,2011-08-03,56 days,set9
9,13101,1639,1637,2011-05-11,2011-06-29,2011-07-13,2011-07-27,56 days,set10


### Function to train the model and get the RMSE values

In [18]:
# function to calculate the root mean squared log error
def get_msle(true, predicted) :
    return np.sqrt(msle(true, predicted))

# function to return the columns on which the model is trained
def get_colums(data):
    print('\n####### The model is trained on Following Columns: ###########\n')
    print(data.columns)
    print('===============================================================')


# function to train the model 
# it will calculate and return the RMSLE on train and validation set    
def my_model(train_d, validate_d, model):    
    train_x = train_d.drop(columns=['WEEK_END_DATE', 'UNITS'])
    train_y = train_d['UNITS']
    
    valid_x = validate_d.drop(columns=['WEEK_END_DATE', 'UNITS'])
    valid_y = validate_d['UNITS']
    
    model.fit(train_x, train_y)
    
    predict_train = model.predict(train_x)
    predict_train = predict_train.clip(min=0)
            
    predict_validate = model.predict(valid_x)
    predict_validate = predict_validate.clip(min=0)
    
    return get_msle(train_y, predict_train), get_msle(valid_y, predict_validate), train_x
    
    
    
# function will extract the train and validation set using validation set dataframe
# The defined model will train on each of the set and the average RMSLE on train and validate set will be returned
def train_model(df, data, model):
    
    model_results_train = []
    model_results_valid = []
    for row in tqdm_notebook(range(df.shape[0]),leave=False, desc='training_model'):
        
        row = df.iloc[row]
        train_set = data[(data.WEEK_END_DATE >= row['train_start']) & (data.WEEK_END_DATE <= row['train_end'])]
        validate_set = data[data.WEEK_END_DATE == row['validate_week']]        
        train, valid, data_train = my_model(train_set,validate_set, model)
        model_results_train.append(train)
        model_results_valid.append(valid)
        
    return np.mean(model_results_train) , np.mean(model_results_valid), data_train

### Feature Generation

#### 1. Rolling Max Sales
- What is the maximum sales in the last the n days? (Here, n = 7)

In [19]:
data_with_unique_store['rolling_sales_max'] = data_with_unique_store.groupby(['STORE_NUM', 'UPC'])['UNITS'].transform(
    lambda x: x.rolling(window=7).max()).astype(np.float16)

In [20]:
# train a model with the new features.
model_RFR = RandomForestRegressor(max_depth=10, n_estimators=175)
new_data = data_with_unique_store.drop(columns=['STORE_NUM', 'UPC'])

valid_df = validation_df(new_data, week, no_of_months=2, no_of_validation= 14)

rmsle_train, rmsle_valid, data_train = train_model(valid_df, new_data, model_RFR)

training_model:   0%|          | 0/14 [00:00<?, ?it/s]

In [21]:
print('RMSLE on train set: ', rmsle_train)
print('RMSLE on validation set:', rmsle_valid)

RMSLE on train set:  0.3552842424923421
RMSLE on validation set: 0.44307856461512957


#### 2. Price Difference of Same Products in Other Stores

In [22]:
df_mean_store = pd.DataFrame(data_with_unique_store.groupby(['UPC', 'WEEK_END_DATE'])['BASE_PRICE'].mean())
df_mean_store.columns = ['BASE_PRICE_AVG']
df_mean_store.reset_index(inplace = True)

In [23]:
df_mean_store.head(5)

,UPC,WEEK_END_DATE,BASE_PRICE_AVG
0,1111009477,2009-01-14,1.281370
1,1111009477,2009-01-21,1.281370
2,1111009477,2009-01-28,1.281096
3,1111009477,2009-02-04,1.282329
4,1111009477,2009-02-11,1.307671


In [24]:
data_with_unique_store = data_with_unique_store.merge(df_mean_store, how= 'left', on= ['UPC', 'WEEK_END_DATE'])

In [25]:
data_with_unique_store['PRICE_DIFF'] = (data_with_unique_store['BASE_PRICE'] - data_with_unique_store['BASE_PRICE_AVG'])/data_with_unique_store['BASE_PRICE_AVG']

In [26]:
# train a model with the new features.
model_RFR = RandomForestRegressor(max_depth=10, n_estimators=175)
new_data = data_with_unique_store.drop(columns=['STORE_NUM', 'UPC'])

valid_df = validation_df(new_data, week, no_of_months=2, no_of_validation= 14)

rmsle_train, rmsle_valid, data_train = train_model(valid_df, new_data, model_RFR)

training_model:   0%|          | 0/14 [00:00<?, ?it/s]

In [27]:
print('RMSLE on train set: ', rmsle_train)
print('RMSLE on validation set:', rmsle_valid)

RMSLE on train set:  0.34416286318399764
RMSLE on validation set: 0.46697055582669605


#### 3. Price difference between week n to week n-1

In [28]:
# Price difference with last week
data_with_unique_store['item_store'] = data_with_unique_store['UPC'].astype(str) + '_' + data_with_unique_store['STORE_NUM'].astype(str)
data_with_unique_store['item_store_change'] = data_with_unique_store["item_store"].shift() != data_with_unique_store["item_store"]

# Price difference week n - week n-1
data_with_unique_store['delta_price_weekn-1'] = (data_with_unique_store['BASE_PRICE']-
                                     data_with_unique_store['BASE_PRICE'].shift(1)).fillna(0)/data_with_unique_store['BASE_PRICE'].shift(1)
data_with_unique_store['delta_price_weekn-1'] = data_with_unique_store['delta_price_weekn-1'].fillna(0) * (data_with_unique_store['item_store_change']==0)

In [29]:
# train a model with the new features.
model_RFR = RandomForestRegressor(max_depth=10, n_estimators=175)
new_data = data_with_unique_store.drop(columns=['STORE_NUM', 'UPC', 'item_store', 'item_store_change'])

valid_df = validation_df(new_data, week, no_of_months=2, no_of_validation= 14)

rmsle_train, rmsle_valid, data_train = train_model(valid_df, new_data, model_RFR)

training_model:   0%|          | 0/14 [00:00<?, ?it/s]

In [30]:
print('RMSLE on train set: ', rmsle_train)
print('RMSLE on validation set:', rmsle_valid)

RMSLE on train set:  0.3442016746872996
RMSLE on validation set: 0.46669324304182924


### Conclusion

- Price Difference between week n to week (n-1) feature has the RMSLE value to 0.4666.
- Price Difference of the same product feature has the RMSLE value to 0.4669.
- Rolling max sales feature has improved the RMSLE value to 0.4430.

`Rolling Max Sales` feature has the more impact on the model as compared to other features.